## RecSys

#### Загрузка датасета, ETL, EDA

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
links = pd.read_csv('C:/Users/rossi/Desktop/MLOps/Books/Dataset/links.csv', sep = ',')
movies = pd.read_csv('C:/Users/rossi/Desktop/MLOps/Books/Dataset/movies.csv', sep = ',')
tags = pd.read_csv('C:/Users/rossi/Desktop/MLOps/Books/Dataset/tags.csv', sep = ',')
ratings = pd.read_csv('C:/Users/rossi/Desktop/MLOps/Books/Dataset/ratings.csv', sep = ',')

In [3]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [4]:
tags = tags.drop(['timestamp'], axis=1) # удаляем ненужную фичу timestamp

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies['genres'].value_counts() # смотрим распределение классов жанров

Drama                                                  1053
Comedy                                                  946
Comedy|Drama                                            435
Comedy|Romance                                          363
Drama|Romance                                           349
                                                       ... 
Action|Crime|Horror|Mystery|Thriller                      1
Adventure|Animation|Children|Comedy|Musical|Romance       1
Action|Adventure|Animation|Comedy|Crime|Mystery           1
Children|Comedy|Fantasy|Sci-Fi                            1
Action|Animation|Comedy|Fantasy                           1
Name: genres, Length: 951, dtype: int64

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
ratings = ratings.drop(['timestamp'], axis=1) # удаляем ненужную фичу timestamp

In [9]:
print(tags.shape)
print(movies.shape)
print(ratings.shape)

(3683, 3)
(9742, 3)
(100836, 3)


#### 1. Простая RecSys, выводящая топ фильмов по рейтингу

In [30]:
rating_count = pd.DataFrame(ratings, columns=['movieId','rating']) # формируем датасет из требуемых данных

In [11]:
rating_count.sort_values('rating', ascending=False).drop_duplicates().head(10) #сортируем топ фильмов по убыванию

,movieId,rating
34031,3147,5.0
50717,1080,5.0
18166,487,5.0
18165,480,5.0
76157,3308,5.0
18158,296,5.0
18157,260,5.0
50696,608,5.0
50698,750,5.0
50699,858,5.0


In [12]:
rating_count.groupby(by = 'rating')['movieId'].max() # группируем фильмы (выводим по 1 фильму в каждой группе оценок рейтинга)

rating
0.5    184641
1.0    190221
1.5    190215
2.0    185033
2.5    189713
3.0    193567
3.5    193587
4.0    193609
4.5    191005
5.0    187717
Name: movieId, dtype: int64

In [13]:
# формируем датасет из 10 любых фильмов в каждой группе

most_rated_books = pd.DataFrame([184641, 190221, 190215, 185033, 189713, 193567, 193587, 193609, 191005, 187717], index=np.arange(10), columns=['movieId']) 
most_rated_books
detail = pd.merge(most_rated_books, movies, on='movieId')
detail

,movieId,title,genres
0,184641,Fullmetal Alchemist 2018 (2017),Action|Adventure|Fantasy
1,190221,Hommage à Zgougou (et salut à Sabine Mamou) (2...,Documentary
2,190215,Liquid Truth (2017),Drama
3,185033,I Kill Giants (2018),Drama|Fantasy|Thriller
4,189713,BlacKkKlansman (2018),Comedy|Crime|Drama
5,193567,anohana: The Flower We Saw That Day - The Movi...,Animation|Drama
6,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
7,193609,Andrew Dice Clay: Dice Rules (1991),Comedy
8,191005,Gintama (2017),Action|Adventure|Comedy|Sci-Fi
9,187717,Won't You Be My Neighbor? (2018),Documentary


In [14]:
# итоговая таблица с подбором фильма в каждой группе оценок

detail2 = pd.merge(detail, rating_count, on='movieId')
detail2

,movieId,title,genres,rating
0,184641,Fullmetal Alchemist 2018 (2017),Action|Adventure|Fantasy,0.5
1,190221,Hommage à Zgougou (et salut à Sabine Mamou) (2...,Documentary,1.0
2,190215,Liquid Truth (2017),Drama,1.5
3,185033,I Kill Giants (2018),Drama|Fantasy|Thriller,2.0
4,189713,BlacKkKlansman (2018),Comedy|Crime|Drama,2.5
5,193567,anohana: The Flower We Saw That Day - The Movi...,Animation|Drama,3.0
6,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,3.5
7,193609,Andrew Dice Clay: Dice Rules (1991),Comedy,4.0
8,191005,Gintama (2017),Action|Adventure|Comedy|Sci-Fi,4.5
9,187717,Won't You Be My Neighbor? (2018),Documentary,5.0


In [68]:
# выводим топ фильмов по рейтингу в каждом жанре

a = pd.merge(ratings, movies, on='movieId')
a.sort_values(by=['rating', 'genres'], ascending=False).drop_duplicates(subset=['title']).head(10)

,userId,movieId,rating,title,genres
23409,57,2921,5.0,High Plains Drifter (1973),Western
31693,240,383,5.0,Wyatt Earp (1994),Western
51374,29,82459,5.0,True Grit (2010),Western
71307,66,3037,5.0,Little Big Man (1970),Western
76105,62,128360,5.0,The Hateful Eight (2015),Western
82609,414,2401,5.0,Pale Rider (1985),Western
82637,66,3074,5.0,Jeremiah Johnson (1972),Western
93295,572,2670,5.0,Run Silent Run Deep (1958),War
2998,1,457,5.0,"Fugitive, The (1993)",Thriller
19278,325,190,5.0,Safe (1995),Thriller


In [64]:
a[a['genres'] == 'Action']['genres'].head(5) # топ-5 фильмов в жанре Action

18826    Action
29524    Action
29525    Action
29526    Action
29527    Action
Name: genres, dtype: object

#### Рекомендашка на основе кореляции Пирсона

In [16]:
ratings['movieId'].value_counts() # смотрим кол-во оценок по каждлму фильму

356       329
318       317
296       307
593       279
2571      278
         ... 
86279       1
86922       1
5962        1
87660       1
163981      1
Name: movieId, Length: 9724, dtype: int64

In [21]:
rat = pd.DataFrame(ratings.groupby('movieId')['rating'].mean()) # посчитаем среднюю оценку по каждому фильму
rat.head()

,rating
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429


In [22]:
rat.sort_values('rating', ascending=False).head() # выводим топ фильмов по рейтингу

,rating
movieId,
88448,5.0
100556,5.0
143031,5.0
143511,5.0
143559,5.0


In [23]:
user_rating= pd.pivot_table(data=ratings, values='rating', index='userId', columns='movieId') # Подготовим таблицу данных для анализа
user_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
correlation_matrix  = user_rating.corr(method='pearson') # Рассчитаем корреляционную матрицу по всему набору книг 
correlation_matrix.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.330978,0.487109,1.000000,0.310971,0.106465,0.208402,0.968246,0.095913,-0.021409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.330978,1.000000,0.419564,NaN,0.562791,0.163510,0.430261,0.415227,0.277350,0.016626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.487109,0.419564,1.000000,NaN,0.602266,0.345069,0.554088,0.333333,0.458591,-0.050276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.000000,NaN,NaN,1.000000,0.654654,NaN,0.203653,NaN,NaN,0.870388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.310971,0.562791,0.602266,0.654654,1.000000,0.291302,0.609119,0.555556,0.319173,0.218263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.106465,0.163510,0.345069,NaN,0.291302,1.000000,-0.123897,-0.801784,-0.172891,0.420222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.208402,0.430261,0.554088,0.203653,0.609119,-0.123897,1.000000,0.816497,0.745356,0.186891,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.968246,0.415227,0.333333,NaN,0.555556,-0.801784,0.816497,1.000000,NaN,0.557086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.095913,0.277350,0.458591,NaN,0.319173,-0.172891,0.745356,NaN,1.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# отберем пользователей, которые оценили конкретную книгу № 10, убедимся, что рейтинг не равен нулю
OneManOut_rating = user_rating[10]
OneManOut_rating[OneManOut_rating>=0]

userId
6      3.0
8      2.0
11     3.0
19     2.0
21     5.0
      ... 
597    3.0
599    3.5
602    3.0
608    4.0
609    4.0
Name: 10, Length: 132, dtype: float64

In [26]:
# оценим сходство с использованием корреляции Пирсона

similar_to_OneManOut = user_rating.corrwith(OneManOut_rating)

corr_OneManOut = pd.DataFrame(similar_to_OneManOut, columns=['PearsonR'])
corr_OneManOut.dropna(inplace=True)
corr_OneManOut.head()

C:\Users\rossi\anaconda3\envs\dzi\lib\site-packages\numpy\lib\function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\rossi\anaconda3\envs\dzi\lib\site-packages\numpy\lib\function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


,PearsonR
movieId,
1,-0.021409
2,0.016626
3,-0.050276
4,0.870388
5,0.218263


In [29]:
OneManOut_corr_summary = corr_OneManOut.join(rat)
# выведем самые похожие книги, отсортировав их при равном сходстве по рейтингу книги
most_rated_books2=OneManOut_corr_summary.sort_values(by = ['PearsonR', 'rating'], ascending=False).head(10)
detail3 = pd.merge(most_rated_books2, movies, on='movieId')
detail3

,movieId,PearsonR,rating,title,genres
0,98154,1.0,4.500000,Lincoln (2012),Drama|War
1,2360,1.0,4.458333,"Celebration, The (Festen) (1998)",Drama
2,7842,1.0,4.375000,Dune (2000),Drama|Fantasy|Sci-Fi
3,176,1.0,4.307692,Living in Oblivion (1995),Comedy
4,26606,1.0,4.250000,"Chinese Ghost Story, A (Sinnui yauwan) (1987)",Action|Fantasy|Horror|Romance
5,84414,1.0,4.250000,All-Star Superman (2011),Action|Adventure|Animation|Sci-Fi
6,2563,1.0,4.166667,Dangerous Beauty (1998),Drama
7,4160,1.0,4.125000,"Widow of St. Pierre, The (Veuve de Saint-Pierr...",Drama|Romance
8,104069,1.0,4.125000,Louis C.K.: Oh My God (2013),Comedy
9,47999,1.0,4.071429,Jesus Camp (2006),Documentary|Drama


#### 3. RecSys на основе содержимого